# IMDB Data Analysis & Wikipedia Stream Processing

## Project Overview

This notebook performs comprehensive analysis of IMDB movie data and implements Wikipedia stream processing.

## Team Members
- SBAI WAHIBA
- HEBBACHE BELKISS

In [6]:
# Import necessary libraries
import os
import gzip
import shutil
import requests
from pathlib import Path
from datetime import datetime

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

import config

print("All imports successful!")

All imports successful!


In [ ]:
print("=" * 80)
print("IMDB DATA ANALYSIS & WIKIPEDIA STREAM PROCESSING")
print("=" * 80)

# Configure Java automatically (if not already set)
import os
java_home = "C:\\Program Files\\Eclipse Adoptium\\jdk-17.0.17.10-hotspot"
if os.path.exists(java_home):
    os.environ['JAVA_HOME'] = java_home
    os.environ['PATH'] = f"{java_home}\\bin;{os.environ.get('PATH', '')}"
    print(f"✓ Java configured: {java_home}")
else:
    print(" Java path not found. Make sure Java is installed.")
    print("   Expected path: C:\\Program Files\\Eclipse Adoptium\\jdk-17.0.17.10-hotspot")

# Create directories
Path(config.DATA_DIR).mkdir(exist_ok=True)
Path(config.IMDB_DIR).mkdir(exist_ok=True)
Path(config.WIKI_STREAM_DIR).mkdir(exist_ok=True)

print(f"Directories created: {config.DATA_DIR}, {config.IMDB_DIR}, {config.WIKI_STREAM_DIR}")

# Initialize Spark Session
try:
    spark = SparkSession.builder \
        .appName("IMDB Analysis") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()
    
    spark.sparkContext.setLogLevel("WARN")
    print("Spark session created successfully!")
    print(f"Spark version: {spark.version}")
except Exception as e:
    print(f"\n ERROR: Could not create Spark session!")
    print(f"Error: {e}")
    print("\n SOLUTION:")
    print("1. Close this notebook")
    print("2. In PowerShell, run:")
    print('   $env:JAVA_HOME = "C:\\Program Files\\Eclipse Adoptium\\jdk-17.0.17.10-hotspot"')
    print('   $env:PATH = "$env:JAVA_HOME\\bin;$env:PATH"')
    print("3. Then start Jupyter from the same PowerShell:")
    print("   jupyter notebook")
    print("4. Reopen this notebook and run this cell again")
    raise

IMDB DATA ANALYSIS & WIKIPEDIA STREAM PROCESSING
✓ Java configured: C:\Program Files\Eclipse Adoptium\jdk-17.0.17.10-hotspot
Directories created: data, data/imdb, data/wiki_stream
Spark session created successfully!
Spark version: 3.5.0


## LOAD DATA FROM IMDB

In [8]:
print("\n" + "=" * 80)
print("QUESTION 1: Loading IMDB Data")
print("=" * 80)

def download_imdb_file(dataset_name, filename):
    """Download IMDB dataset file"""
    url = f"{config.IMDB_BASE_URL}{filename}"
    filepath = f"{config.IMDB_DIR}/{filename}"
    
    # Skip if file already exists
    if os.path.exists(filepath.replace('.gz', '')):
        print(f"{dataset_name} already exists, skipping download")
        return filepath.replace('.gz', '')
    
    if os.path.exists(filepath):
        print(f"{dataset_name} archive exists, extracting...")
    else:
        print(f"Downloading {dataset_name}...")
        try:
            response = requests.get(url, stream=True, timeout=300)
            response.raise_for_status()
            
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Downloaded {dataset_name}")
        except Exception as e:
            print(f"Error downloading {dataset_name}: {e}")
            print(f"\n*** MANUAL DOWNLOAD REQUIRED ***")
            print(f"Please download {filename} from {url}")
            print(f"Place it in {config.IMDB_DIR}/ and extract if needed")
            return None
    
    # Extract if gzipped
    if filename.endswith('.gz'):
        extracted_path = filepath.replace('.gz', '')
        if not os.path.exists(extracted_path):
            print(f"Extracting {dataset_name}...")
            try:
                with gzip.open(filepath, 'rb') as f_in:
                    with open(extracted_path, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                print(f"Extracted {dataset_name}")
            except Exception as e:
                print(f"Error extracting {dataset_name}: {e}")
                return None
        return extracted_path
    
    return filepath

# Download all IMDB datasets
downloaded_files = {}
manual_files = []

for dataset_name, filename in config.IMDB_DATASETS.items():
    filepath = download_imdb_file(dataset_name, filename)
    if filepath:
        downloaded_files[dataset_name] = filepath
    else:
        manual_files.append((dataset_name, filename))

print(f"\nSuccessfully downloaded: {list(downloaded_files.keys())}")
if manual_files:
    print(f"\n*** Manual download required for: {[f[0] for f in manual_files]} ***")

# Load datasets into Spark DataFrames
def load_tsv(filepath):
    """Load TSV file into DataFrame"""
    if not os.path.exists(filepath):
        print(f"File not found: {filepath}")
        return None
    
    df = spark.read.option("sep", "\t") \
        .option("header", "true") \
        .option("nullValue", "\\N") \
        .option("inferSchema", "true") \
        .csv(filepath)
    
    return df

# Load all available datasets
name_basics = None
title_basics = None
title_ratings = None
title_crew = None
title_principals = None
title_akas = None

if 'name_basics' in downloaded_files:
    name_basics = load_tsv(downloaded_files['name_basics'])
    print(f"Loaded name_basics: {name_basics.count():,} records")

if 'title_basics' in downloaded_files:
    title_basics = load_tsv(downloaded_files['title_basics'])
    print(f"Loaded title_basics: {title_basics.count():,} records")

if 'title_ratings' in downloaded_files:
    title_ratings = load_tsv(downloaded_files['title_ratings'])
    print(f"Loaded title_ratings: {title_ratings.count():,} records")

if 'title_crew' in downloaded_files:
    title_crew = load_tsv(downloaded_files['title_crew'])
    print(f"Loaded title_crew: {title_crew.count():,} records")

if 'title_principals' in downloaded_files:
    title_principals = load_tsv(downloaded_files['title_principals'])
    print(f"Loaded title_principals: {title_principals.count():,} records")

if 'title_akas' in downloaded_files:
    title_akas = load_tsv(downloaded_files['title_akas'])
    print(f"Loaded title_akas: {title_akas.count():,} records")

print("\nAll datasets loaded successfully!")


QUESTION 1: Loading IMDB Data
Downloaded name_basics
Extracting name_basics...
Extracted name_basics
Downloaded title_akas
Extracting title_akas...
Extracted title_akas
Downloaded title_basics
Extracting title_basics...
Extracted title_basics
Downloaded title_crew
Extracting title_crew...
Extracted title_crew
Downloaded title_episode
Extracting title_episode...
Extracted title_episode
Downloaded title_principals
Extracting title_principals...
Extracted title_principals
Downloaded title_ratings
Extracting title_ratings...
Extracted title_ratings

Successfully downloaded: ['name_basics', 'title_akas', 'title_basics', 'title_crew', 'title_episode', 'title_principals', 'title_ratings']
Loaded name_basics: 14,942,131 records
Loaded title_basics: 12,142,742 records
Loaded title_ratings: 1,608,088 records
Loaded title_crew: 12,144,623 records
Loaded title_principals: 96,550,019 records
Loaded title_akas: 54,383,161 records

All datasets loaded successfully!


## How many total people in dataset?

In [9]:
print("\n" + "=" * 80)
print("QUESTION 2: How many total people in dataset?")
print("=" * 80)

if name_basics is not None:
    total_people = name_basics.count()
    print(f"Total people in dataset: {total_people:,}")
else:
    print("name_basics dataset not loaded")
    total_people = None


QUESTION 2: How many total people in dataset?
Total people in dataset: 14,942,131


## What is the earliest year of birth?

In [10]:
print("\n" + "=" * 80)
print("QUESTION 3: What is the earliest year of birth?")
print("=" * 80)

if name_basics is not None:
    earliest_birth = name_basics.select("birthYear") \
        .filter(col("birthYear").isNotNull()) \
        .agg(min("birthYear").alias("earliest_birth")) \
        .collect()[0]["earliest_birth"]
    
    print(f"Earliest year of birth: {earliest_birth}")
else:
    print("name_basics dataset not loaded")
    earliest_birth = None


QUESTION 3: What is the earliest year of birth?
Earliest year of birth: 4


## How many years ago was this person born?

In [11]:
print("\n" + "=" * 80)
print("QUESTION 4: How many years ago was this person born?")
print("=" * 80)

if name_basics is not None and earliest_birth is not None:
    current_year = datetime.now().year
    years_ago = current_year - earliest_birth
    
    # Get person details
    earliest_person = name_basics.filter(col("birthYear") == earliest_birth) \
        .select("nconst", "primaryName", "birthYear") \
        .first()
    
    print(f"Earliest birth year: {earliest_birth}")
    print(f"Current year: {current_year}")
    print(f"Years ago: {years_ago:,} years")
    if earliest_person:
        print(f"Person: {earliest_person['primaryName']} (ID: {earliest_person['nconst']})")
else:
    print("Cannot calculate - data not available")
    earliest_person = None
    years_ago = None


QUESTION 4: How many years ago was this person born?
Earliest birth year: 4
Current year: 2025
Years ago: 2,021 years
Person: Lucio Anneo Seneca (ID: nm0784172)


In [24]:
print("\n" + "=" * 80)
print("QUESTION 5: Using only the data in the dataset, determine if this date of birth is correct.")
print("=" * 80)

if name_basics is not None and earliest_birth is not None:
    # Get person with earliest birth year
    earliest_person_df = name_basics.filter(col("birthYear") == earliest_birth) \
        .select("nconst", "primaryName", "birthYear", "deathYear", "primaryProfession", "knownForTitles")
    
    earliest_person = earliest_person_df.first()
    
    print(f"Person: {earliest_person['primaryName']}")
    print(f"Birth Year: {earliest_person['birthYear']}")
    print(f"Death Year: {earliest_person['deathYear']}")
    print(f"Primary Profession: {earliest_person['primaryProfession']}")
    print(f"Known For Titles: {earliest_person['knownForTitles']}")
    
    # Validation logic (using only the data in the dataset)
    validation_result = "UNKNOWN"
    reasoning = []
    
    # Special check: Very old dates (before year 100) might be BCE dates without notation
    bce_warning = False
    if earliest_person['birthYear'] < 100:
        bce_warning = True
    
    if earliest_person['deathYear']:
        lifespan = earliest_person['deathYear'] - earliest_person['birthYear']
        reasoning.append(f"Lifespan: {lifespan} years")
        
        if lifespan > 150:
            validation_result = "LIKELY INCORRECT"
            reasoning.append("Lifespan exceeds 150 years, which is extremely unlikely")
        elif lifespan < 0:
            validation_result = "INCORRECT"
            reasoning.append("Death year before birth year - impossible")
        elif lifespan > 120:
            validation_result = "POSSIBLY INCORRECT"
            reasoning.append("Lifespan exceeds 120 years, which is very rare")
        elif bce_warning:
            # If birth year is very old, mark as potentially incorrect due to BCE/CE issue
            validation_result = "POSSIBLY INCORRECT (BCE/CE notation missing)"
            reasoning.append(f"Lifespan of {lifespan} years calculated, but birth year may be BCE")
            reasoning.append("Historical records needed to verify correct dating")
        else:
            validation_result = "PLAUSIBLE"
            reasoning.append(f"Lifespan of {lifespan} years is within reasonable range")
    
    # Check known for titles if available
    if earliest_person['knownForTitles']:
        title_ids = earliest_person['knownForTitles'].split(",")
        reasoning.append(f"Known for {len(title_ids)} titles")
        reasoning.append("Title years could be cross-referenced for validation")
    
    print(f"\n" + "-" * 80)
    print("QUESTION 5 ANSWER (Using only the data in the dataset):")
    print("-" * 80)
    print(f"Based on the dataset analysis, the date of birth is: {validation_result}")
    
    # QUESTION 6: Explain the reasoning for the answer
    print(f"\n" + "=" * 80)
    print("QUESTION 6: Explain the reasoning for the answer")
    print("=" * 80)
    print("\nAnalysis using only dataset information:")
    for reason in reasoning:
        print(f"  - {reason}")
    print("\n" + "-" * 80)
    print("REASONING:")
    print("-" * 80)
    print("""
The validation of the earliest birth date is based on:

1. **Lifespan Analysis**: If a death year is recorded, we calculate the lifespan. 
   Lifespans exceeding 150 years are considered extremely unlikely, while those 
   over 120 are flagged as possibly incorrect.

2. **Logical Consistency**: We check if death year is before birth year, which 
   would be logically impossible.

3. **Cross-Reference with Titles**: If the person has known titles, we could 
   potentially cross-reference with title release years, though this requires 
   additional joins.

**PROBLEM WITH THIS SPECIFIC CASE:**

Lucio Anneo Seneca (Seneca the Younger) was actually born in **4 BCE (Before Common Era)**, 
not in year 4 CE. The dataset stores it as "4" without indicating BCE/CE.

- **Correct**: Born 4 BCE, died 65 CE → Lifespan = ~69 years (4 + 65 - 1 = 68-69 years)
- **Dataset calculation**: 65 - 4 = 61 years
- **Conclusion**: The calculation is mathematically correct based on the dataset, but 
  historically incorrect because it doesn't account for BCE/CE notation.

**Limitation**: Without external historical records or BCE/CE indicators in the dataset, 
we can only assess plausibility based on internal consistency. Very old dates (especially 
before year 100) may represent BCE dates stored without proper notation, leading to 
incorrect lifespan calculations.

**Final Answer**: The date validation method is logically sound, but in this specific 
case, the result is historically incorrect due to missing BCE/CE notation in the dataset.
""")
    
    final_validation_answer = validation_result
else:
    print("Cannot validate - data not available")
    final_validation_answer = "Cannot determine"



QUESTION 5: Using only the data in the dataset, determine if this date of birth is correct.
Person: Lucio Anneo Seneca
Birth Year: 4
Death Year: 65
Primary Profession: writer
Known For Titles: tt0043802,tt0218822,tt0049203,tt0972562

--------------------------------------------------------------------------------
QUESTION 5 ANSWER (Using only the data in the dataset):
--------------------------------------------------------------------------------
Based on the dataset analysis, the date of birth is: POSSIBLY INCORRECT (BCE/CE notation missing)

QUESTION 6: Explain the reasoning for the answer

Analysis using only dataset information:
  - Lifespan: 61 years
  - Lifespan of 61 years calculated, but birth year may be BCE
  - Historical records needed to verify correct dating
  - Known for 4 titles
  - Title years could be cross-referenced for validation

--------------------------------------------------------------------------------
REASONING:
-------------------------------------------

## What is the most recent date of birth?

In [25]:
print("\n" + "=" * 80)
print("QUESTION 7: What is the most recent date of birth?")
print("=" * 80)

if name_basics is not None:
    most_recent_birth = name_basics.select("birthYear") \
        .filter(col("birthYear").isNotNull()) \
        .agg(max("birthYear").alias("most_recent_birth")) \
        .collect()[0]["most_recent_birth"]
    
    print(f"Most recent birth year: {most_recent_birth}")
    
    # Get person details
    recent_person = name_basics.filter(col("birthYear") == most_recent_birth) \
        .select("nconst", "primaryName", "birthYear") \
        .first()
    
    if recent_person:
        print(f"Person: {recent_person['primaryName']} (ID: {recent_person['nconst']})")
else:
    print("name_basics dataset not loaded")
    most_recent_birth = None


QUESTION 7: What is the most recent date of birth?
Most recent birth year: 2025
Person: Kyrah Ivy Jackson (ID: nm16784939)


## What percentage of people do not have a listed date of birth?

In [26]:
print("\n" + "=" * 80)
print("QUESTION 8: What percentage of people do not have a listed date of birth?")
print("=" * 80)

if name_basics is not None:
    total_count = name_basics.count()
    without_birth = name_basics.filter(col("birthYear").isNull()).count()
    percentage = (without_birth / total_count) * 100
    
    print(f"Total people: {total_count:,}")
    print(f"People without birth year: {without_birth:,}")
    print(f"Percentage without birth year: {percentage:.2f}%")
else:
    print("name_basics dataset not loaded")
    percentage = None


QUESTION 8: What percentage of people do not have a listed date of birth?
Total people: 14,942,131
People without birth year: 14,281,170
Percentage without birth year: 95.58%


## What is the length of the longest "short" after 1900?

In [27]:
print("\n" + "=" * 80)
print("QUESTION 9: What is the length of the longest \"short\" after 1900?")
print("=" * 80)

if title_basics is not None:
    longest_short = title_basics.filter(
        (col("titleType") == "short") & 
        (col("startYear") > 1900) & 
        (col("runtimeMinutes").isNotNull())
    ).select("tconst", "primaryTitle", "titleType", "startYear", "runtimeMinutes") \
     .orderBy(desc("runtimeMinutes")) \
     .first()
    
    if longest_short:
        print(f"Longest 'short' after 1900:")
        print(f"  Title: {longest_short['primaryTitle']}")
        print(f"  Year: {longest_short['startYear']}")
        print(f"  Runtime: {longest_short['runtimeMinutes']} minutes")
        print(f"  ID: {longest_short['tconst']}")
    else:
        print("No shorts found after 1900 with runtime information")
        longest_short = None
else:
    print("title_basics dataset not loaded")
    longest_short = None


QUESTION 9: What is the length of the longest "short" after 1900?
Longest 'short' after 1900:
  Title: Bella
  Year: 2021
  Runtime: 97 minutes
  ID: tt10699260


## What is the length of the shortest "movie" after 1900?

In [28]:
print("\n" + "=" * 80)
print("QUESTION 10: What is the length of the shortest \"movie\" after 1900?")
print("=" * 80)

if title_basics is not None:
    shortest_movie = title_basics.filter(
        (col("titleType") == "movie") & 
        (col("startYear") > 1900) & 
        (col("runtimeMinutes").isNotNull()) & 
        (col("runtimeMinutes") > 0)
    ).select("tconst", "primaryTitle", "titleType", "startYear", "runtimeMinutes") \
     .orderBy(asc("runtimeMinutes")) \
     .first()
    
    if shortest_movie:
        print(f"Shortest 'movie' after 1900:")
        print(f"  Title: {shortest_movie['primaryTitle']}")
        print(f"  Year: {shortest_movie['startYear']}")
        print(f"  Runtime: {shortest_movie['runtimeMinutes']} minutes")
        print(f"  ID: {shortest_movie['tconst']}")
    else:
        print("No movies found after 1900 with runtime information")
        shortest_movie = None
else:
    print("title_basics dataset not loaded")
    shortest_movie = None


QUESTION 10: What is the length of the shortest "movie" after 1900?
Shortest 'movie' after 1900:
  Title: Dancing Boy
  Year: 2023
  Runtime: 1 minutes
  ID: tt26348770


## List all of the genres represented.

In [29]:
print("\n" + "=" * 80)
print("QUESTION 11: List all of the genres represented.")
print("=" * 80)

if title_basics is not None:
    # Split genres and explode to get individual genres
    genres_df = title_basics.select("genres") \
        .filter(col("genres").isNotNull()) \
        .select(explode(split(col("genres"), "\\|")).alias("genre")) \
        .distinct() \
        .orderBy("genre")
    
    all_genres = [row.genre for row in genres_df.collect()]
    
    print(f"Total unique genres: {len(all_genres)}")
    print(f"\nAll genres:")
    for genre in all_genres:
        print(f"  - {genre}")
else:
    print("title_basics dataset not loaded")
    all_genres = []


QUESTION 11: List all of the genres represented.
Total unique genres: 2377

All genres:
  - Action
  - Action,Adult
  - Action,Adult,Adventure
  - Action,Adult,Animation
  - Action,Adult,Comedy
  - Action,Adult,Crime
  - Action,Adult,Documentary
  - Action,Adult,Drama
  - Action,Adult,Fantasy
  - Action,Adult,History
  - Action,Adult,Horror
  - Action,Adult,Romance
  - Action,Adult,Sci-Fi
  - Action,Adult,Short
  - Action,Adult,Sport
  - Action,Adult,Thriller
  - Action,Adult,War
  - Action,Adult,Western
  - Action,Adventure
  - Action,Adventure,Animation
  - Action,Adventure,Biography
  - Action,Adventure,Comedy
  - Action,Adventure,Crime
  - Action,Adventure,Documentary
  - Action,Adventure,Drama
  - Action,Adventure,Family
  - Action,Adventure,Fantasy
  - Action,Adventure,Game-Show
  - Action,Adventure,History
  - Action,Adventure,Horror
  - Action,Adventure,Music
  - Action,Adventure,Musical
  - Action,Adventure,Mystery
  - Action,Adventure,News
  - Action,Adventure,Reality-TV
  -

## What is the highest rated comedy "movie"?

In [33]:
print("\n" + "=" * 80)
print("QUESTION 12: What is the highest rated comedy \"movie\"?")
print("(Tie broken by most votes)")
print("=" * 80)

best_comedy_tconst = None

if title_basics is not None and title_ratings is not None:
    # Join title_basics with title_ratings
    movies_with_ratings = title_basics.join(
        title_ratings, 
        on="tconst", 
        how="inner"
    )
    
    # Filter for comedy movies
    comedy_movies = movies_with_ratings.filter(
        (col("titleType") == "movie") & 
        (col("genres").contains("Comedy"))
    )
    
    # Order by rating (desc) then votes (desc) to break ties
    highest_rated_comedy = comedy_movies.select(
        "tconst", 
        "primaryTitle", 
        "startYear", 
        "averageRating", 
        "numVotes",
        "genres"
    ).orderBy(desc("averageRating"), desc("numVotes")) \
     .first()
    
    if highest_rated_comedy:
        print(f"Highest rated comedy movie:")
        print(f"  Title: {highest_rated_comedy['primaryTitle']}")
        print(f"  Year: {highest_rated_comedy['startYear']}")
        print(f"  Rating: {highest_rated_comedy['averageRating']}")
        print(f"  Votes: {highest_rated_comedy['numVotes']:,}")
        print(f"  Genres: {highest_rated_comedy['genres']}")
        print(f"  ID: {highest_rated_comedy['tconst']}")
        
        best_comedy_tconst = highest_rated_comedy['tconst']
    else:
        print("No comedy movies found")
else:
    print("Required datasets not loaded")


QUESTION 12: What is the highest rated comedy "movie"?
(Tie broken by most votes)
Highest rated comedy movie:
  Title: O La La
  Year: 2018
  Rating: 10.0
  Votes: 6
  Genres: Comedy
  ID: tt8458418


## Who was the director of the movie?

In [34]:
print("\n" + "=" * 80)
print("QUESTION 13: Who was the director of the movie?")
print("=" * 80)

movie_director = "Unknown"

if best_comedy_tconst and title_crew is not None and name_basics is not None:
    # Get directors for the movie
    crew_info = title_crew.filter(col("tconst") == best_comedy_tconst) \
        .select("directors") \
        .first()
    
    if crew_info and crew_info['directors']:
        director_ids = crew_info['directors'].split(",")
        
        # Get director names
        directors = name_basics.filter(col("nconst").isin(director_ids)) \
            .select("primaryName") \
            .collect()
        
        director_names = [d['primaryName'] for d in directors]
        
        print(f"Director(s) of the highest rated comedy movie:")
        for director in director_names:
            print(f"  - {director}")
        
        if len(director_names) == 1:
            movie_director = director_names[0]
        else:
            movie_director = ", ".join(director_names)
    else:
        print("No director information found for this movie")
else:
    print("Cannot find director - required data not available")


QUESTION 13: Who was the director of the movie?
Director(s) of the highest rated comedy movie:
  - Sripad Pai


## List alternate titles for the movie.

In [36]:
print("\n" + "=" * 80)
print("QUESTION 14: List alternate titles for the movie.")
print("=" * 80)

if best_comedy_tconst and title_akas is not None:
    alternate_titles = title_akas.filter(col("titleId") == best_comedy_tconst) \
        .select("title", "region", "language", "types", "attributes", "isOriginalTitle") \
        .orderBy("region", "language")
    
    alt_titles_list = alternate_titles.collect()
    
    if alt_titles_list:
        # Extract unique title texts (actual alternate titles with different text)
        unique_titles = {}
        for title_entry in alt_titles_list:
            title_text = title_entry['title']
            if title_text not in unique_titles:
                unique_titles[title_text] = []
            unique_titles[title_text].append(title_entry)
        
        if len(unique_titles) > 1:
            # There are actual alternate titles with different text
            print("Alternate titles:")
            for i, (title_text, entries) in enumerate(unique_titles.items(), 1):
                print(f"  {i}. \"{title_text}\"")
                # Show one example entry for this title
                if entries:
                    example = entries[0]
                    print(f"     (Region: {example['region'] or 'N/A'}, Language: {example['language'] or 'N/A'})")
        else:
            # Only one unique title text - no alternate titles, just regional variants
            print(f"\nThe dataset contains {len(alt_titles_list)} regional/language variant(s) of the same title:")
            for entry in alt_titles_list:
                print(f"  - \"{entry['title']}\" (Region: {entry['region'] or 'N/A'}, Language: {entry['language'] or 'N/A'}, Type: {entry['types'] or 'N/A'})")
    else:
        print("No alternate titles found for this movie")
else:
    print("Cannot find alternate titles - required data not available")


QUESTION 14: List alternate titles for the movie.

The dataset contains 2 regional/language variant(s) of the same title:
  - "O La La" (Region: N/A, Language: N/A, Type: original)
  - "O La La" (Region: IN, Language: en, Type: imdbDisplay)


## WIKIPEDIA STREAM PROCESSING

In [39]:
print("\n" + "=" * 80)
print("PART 2: WIKIPEDIA STREAM PROCESSING")
print("=" * 80)

# Import additional libraries for stream processing
import json
import time
from datetime import datetime
from collections import defaultdict

print("""
Overview:
---------
This section implements a stream processing job that tracks Wikipedia events 
for selected IMDB entities. The system:

1. Selects 5 entities from the IMDB dataset
2. Monitors Wikipedia via EventStreams API for changes to these entities
3. Generates metrics such as edit counts, user activity, change frequency
4. Implements alerts for specific events (e.g., vandalism, deletions, blocks)
5. Stores results in structured format (JSON files)

Output Structure:
-----------------
- Metrics: stored in data/wiki_stream/metrics/ (JSON format)
- Alerts: stored in data/wiki_stream/alerts/ (JSON format, separate from metrics)

Note: This implementation collects a sample of events for demonstration.
In production, this would run continuously as a streaming job.
""")



# Select entities to track
print("\nSelecting entities to track...")

entities_to_track = []

# 1. Movie (highest rated comedy)
if best_comedy_tconst and title_basics is not None:
    movie_title = title_basics.filter(col("tconst") == best_comedy_tconst) \
        .select("primaryTitle").first()['primaryTitle']
    entities_to_track.append({
        'type': 'movie',
        'name': movie_title,
        'id': best_comedy_tconst
    })

# 2. Popular actor
if title_principals is not None and name_basics is not None:
    popular_actor = title_principals.filter(col("category") == "actor") \
        .groupBy("nconst") \
        .count() \
        .orderBy(desc("count")) \
        .limit(1) \
        .join(name_basics, on="nconst") \
        .select("primaryName", "nconst") \
        .first()
    
    if popular_actor:
        entities_to_track.append({
            'type': 'actor',
            'name': popular_actor['primaryName'],
            'id': popular_actor['nconst']
        })

# 3. Genre
entities_to_track.append({
    'type': 'genre',
    'name': 'Comedy',
    'id': 'genre_comedy'
})

# 4. Director
if movie_director and movie_director != "Unknown":
    entities_to_track.append({
        'type': 'director',
        'name': movie_director.split(',')[0].strip(),
        'id': 'director_' + movie_director.split(',')[0].strip().replace(' ', '_')
    })

# 5. Popular title (most voted movie)
if title_ratings is not None and title_basics is not None:
    popular_title = title_basics.join(title_ratings, on="tconst") \
        .filter(col("titleType") == "movie") \
        .orderBy(desc("numVotes")) \
        .limit(1) \
        .select("primaryTitle", "tconst") \
        .first()
    
    if popular_title:
        entities_to_track.append({
            'type': 'movie',
            'name': popular_title['primaryTitle'],
            'id': popular_title['tconst']
        })

print(f"\nSelected {len(entities_to_track)} entities to track:")
for i, entity in enumerate(entities_to_track, 1):
    print(f"{i}. {entity['type'].upper()}: {entity['name']} (ID: {entity['id']})")

# ============================================================================
# STREAM PROCESSING IMPLEMENTATION
# ============================================================================

print("\n" + "-" * 80)
print("STEP 1: Helper Functions for Stream Processing")
print("-" * 80)

def normalize_title(title):
    """Normalize Wikipedia title for matching"""
    return title.replace("_", " ").strip().lower()

def collect_wiki_events(entities, duration_seconds=30, max_events=50):
    """Collect Wikipedia events from EventStreams API"""
    events = []
    entity_names = [normalize_title(e['name']) for e in entities]
    
    print(f"\nConnecting to Wikipedia EventStreams API...")
    print(f"Tracking entities: {[e['name'] for e in entities]}")
    print(f"Collecting events for {duration_seconds} seconds (max {max_events} events)...")
    
    try:
        url = config.WIKI_STREAM_URL
        response = requests.get(url, stream=True, timeout=duration_seconds + 5)
        response.raise_for_status()
        
        start_time = time.time()
        event_count = 0
        
        for line in response.iter_lines():
            if time.time() - start_time > duration_seconds or event_count >= max_events:
                break
                
            if line:
                try:
                    event = json.loads(line)
                    page_title = event.get('title', '')
                    normalized_page = normalize_title(page_title)
                    
                    # Check if event matches any tracked entity
                    matched_entity = None
                    for entity in entities:
                        entity_name_normalized = normalize_title(entity['name'])
                        if entity_name_normalized in normalized_page or normalized_page in entity_name_normalized:
                            matched_entity = entity
                            break
                    
                    if matched_entity:
                        event['matched_entity'] = matched_entity
                        events.append(event)
                        event_count += 1
                        print(f"  ✓ Event {event_count}: {page_title} ({event.get('type', 'unknown')})")
                        
                except json.JSONDecodeError:
                    continue
                    
    except Exception as e:
        print(f"Note: Connection issue - {e}")
        print("Using simulated events for demonstration purposes.")
        # Create sample events for demonstration
        for entity in entities[:2]:  # Use first 2 entities for demo
            events.append({
                'title': entity['name'],
                'type': 'edit',
                'user': 'SampleUser',
                'bot': False,
                'timestamp': datetime.now().isoformat(),
                'length': {'old': 5000, 'new': 5500},
                'comment': 'Sample edit',
                'matched_entity': entity
            })
    
    print(f"\nTotal events collected: {len(events)}")
    return events

def process_events_for_metrics(events):
    """Process events to calculate metrics"""
    metrics = {
        'total_events': len(events),
        'events_by_type': defaultdict(int),
        'events_by_entity': defaultdict(int),
        'events_by_user': defaultdict(int),
        'bot_edits': 0,
        'human_edits': 0,
        'change_sizes': [],
        'event_timestamps': []
    }
    
    for event in events:
        event_type = event.get('type', 'unknown')
        metrics['events_by_type'][event_type] += 1
        
        if 'matched_entity' in event:
            entity_name = event['matched_entity']['name']
            metrics['events_by_entity'][entity_name] += 1
        
        user = event.get('user', 'unknown')
        metrics['events_by_user'][user] += 1
        
        if event.get('bot', False):
            metrics['bot_edits'] += 1
        else:
            metrics['human_edits'] += 1
        
        length = event.get('length', {})
        old_len = length.get('old', 0)
        new_len = length.get('new', 0)
        if old_len > 0 and new_len > 0:
            change_size = new_len - old_len
            metrics['change_sizes'].append(change_size)
        
        if 'timestamp' in event:
            metrics['event_timestamps'].append(event['timestamp'])
    
    return metrics

def detect_alerts(events):
    """Detect alert-type events that require action"""
    alerts = []
    alert_keywords = ['vandalism', 'deletion', 'block', 'revert', 'protection']
    
    for event in events:
        alert = None
        
        # Check comment for alert keywords
        comment = event.get('comment', '').lower()
        for keyword in alert_keywords:
            if keyword in comment:
                alert = {
                    'timestamp': event.get('timestamp', datetime.now().isoformat()),
                    'entity': event.get('matched_entity', {}).get('name', 'unknown'),
                    'alert_type': keyword.upper(),
                    'user': event.get('user', 'unknown'),
                    'details': comment,
                    'severity': 'HIGH' if keyword in ['vandalism', 'block'] else 'MEDIUM'
                }
                break
        
        # Check for large deletions (>1000 bytes removed)
        if not alert:
            length = event.get('length', {})
            old_len = length.get('old', 0)
            new_len = length.get('new', 0)
            if old_len > 0 and new_len > 0:
                deletion_size = old_len - new_len
                if deletion_size > 1000:
                    alert = {
                        'timestamp': event.get('timestamp', datetime.now().isoformat()),
                        'entity': event.get('matched_entity', {}).get('name', 'unknown'),
                        'alert_type': 'LARGE_DELETION',
                        'user': event.get('user', 'unknown'),
                        'details': f'Large deletion: {deletion_size} bytes removed',
                        'severity': 'MEDIUM'
                    }
        
        if alert:
            alerts.append(alert)
    
    return alerts

# ============================================================================
# STEP 2: Collect Events from Wikipedia EventStreams
# ============================================================================

print("\n" + "-" * 80)
print("STEP 2: Collecting Events from Wikipedia EventStreams")
print("-" * 80)

# Collect events (30 seconds, max 50 events for demo)
events = collect_wiki_events(entities_to_track, duration_seconds=30, max_events=50)

# ============================================================================
# STEP 3: Process Events and Calculate Metrics
# ============================================================================

print("\n" + "-" * 80)
print("STEP 3: Processing Events and Calculating Metrics")
print("-" * 80)

metrics = process_events_for_metrics(events)

print(f"\nMetrics Summary:")
print(f"  Total events: {metrics['total_events']}")
print(f"  Events by type: {dict(metrics['events_by_type'])}")
print(f"  Events by entity: {dict(metrics['events_by_entity'])}")
print(f"  Bot edits: {metrics['bot_edits']}")
print(f"  Human edits: {metrics['human_edits']}")
if metrics['change_sizes']:
    # Use Python's built-in sum (not PySpark's sum function)
    total_change = __builtins__.sum(metrics['change_sizes'])
    avg_change = total_change / len(metrics['change_sizes'])
    print(f"  Avg change size: {avg_change:.0f} bytes")

# ============================================================================
# STEP 4: Detect Alerts
# ============================================================================

print("\n" + "-" * 80)
print("STEP 4: Detecting Alerts")
print("-" * 80)

alerts = detect_alerts(events)
print(f"\nTotal alerts detected: {len(alerts)}")
if alerts:
    for i, alert in enumerate(alerts, 1):
        print(f"  Alert {i}: {alert['alert_type']} - {alert['entity']} (Severity: {alert['severity']})")

# ============================================================================
# STEP 5: Save Metrics to File
# ============================================================================

print("\n" + "-" * 80)
print("STEP 5: Saving Metrics to File")
print("-" * 80)

metrics_dir = Path(config.WIKI_STREAM_DIR) / "metrics"
metrics_dir.mkdir(parents=True, exist_ok=True)

metrics_file = metrics_dir / f"metrics_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

metrics_output = {
    'total_events': metrics['total_events'],
    'events_by_type': dict(metrics['events_by_type']),
    'events_by_entity': dict(metrics['events_by_entity']),
    'events_by_user': dict(metrics['events_by_user']),
    'bot_edits': metrics['bot_edits'],
    'human_edits': metrics['human_edits'],
    'change_sizes': metrics['change_sizes'],
    'summary': {
        'total_events': metrics['total_events'],
        'unique_entities': len(metrics['events_by_entity']),
        'unique_users': len(metrics['events_by_user']),
        'bot_ratio': metrics['bot_edits'] / __builtins__.max(metrics['total_events'], 1),
        'avg_change_size': __builtins__.sum(metrics['change_sizes']) / len(metrics['change_sizes']) if metrics['change_sizes'] else 0
    },
    'schema': {
        'description': 'Wikipedia stream processing metrics',
        'fields': [
            'total_events: Integer - Total number of events collected',
            'events_by_type: Dict - Count of events grouped by type (edit, new, etc.)',
            'events_by_entity: Dict - Count of events grouped by tracked entity',
            'events_by_user: Dict - Count of events grouped by user',
            'bot_edits: Integer - Number of bot edits',
            'human_edits: Integer - Number of human edits',
            'change_sizes: List[Integer] - List of change sizes in bytes',
            'summary: Dict - Summary statistics'
        ]
    }
}

with open(metrics_file, 'w') as f:
    json.dump(metrics_output, f, indent=2)

print(f"Metrics saved to: {metrics_file}")

# ============================================================================
# STEP 6: Save Alerts to Separate File (Alert System)
# ============================================================================

print("\n" + "-" * 80)
print("STEP 6: Saving Alerts to Separate File (Alert System)")
print("-" * 80)

alerts_dir = Path(config.WIKI_STREAM_DIR) / "alerts"
alerts_dir.mkdir(parents=True, exist_ok=True)

alerts_file = alerts_dir / f"alerts_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

alerts_by_type = defaultdict(int)
alerts_by_severity = defaultdict(int)

for alert in alerts:
    alerts_by_type[alert['alert_type']] += 1
    alerts_by_severity[alert['severity']] += 1

alerts_output = {
    'total_alerts': len(alerts),
    'alerts': alerts,
    'alerts_by_type': dict(alerts_by_type),
    'alerts_by_severity': dict(alerts_by_severity),
    'schema': {
        'description': 'Wikipedia stream processing alerts - events requiring action',
        'fields': [
            'timestamp: String - ISO timestamp of alert',
            'entity: String - Entity name that triggered alert',
            'alert_type: String - Type of alert (VANDALISM, DELETION, BLOCK, etc.)',
            'user: String - Wikipedia username',
            'details: String - Alert details/comment',
            'severity: String - Severity level (HIGH, MEDIUM, LOW)'
        ]
    }
}

with open(alerts_file, 'w') as f:
    json.dump(alerts_output, f, indent=2)

print(f"Alerts saved to: {alerts_file}")

# ============================================================================
# STEP 7: Create Documentation
# ============================================================================

print("\n" + "-" * 80)
print("STEP 7: Creating Documentation")
print("-" * 80)

stream_doc = f"""
# Wikipedia Stream Processing Output Structure

## Selected Entities
{chr(10).join([f"- {e['type'].upper()}: {e['name']} (ID: {e['id']})" for e in entities_to_track])}

## Metrics Storage
**Location**: `{config.WIKI_STREAM_DIR}/metrics/`

**Format**: JSON files (one file per collection session)

**Schema**:
- total_events: Integer - Total number of events collected
- events_by_type: Dict - Count of events grouped by type
- events_by_entity: Dict - Count of events grouped by tracked entity
- events_by_user: Dict - Count of events grouped by Wikipedia user
- bot_edits: Integer - Number of bot edits
- human_edits: Integer - Number of human edits
- change_sizes: List[Integer] - List of change sizes in bytes
- summary: Dict - Summary statistics (unique entities, users, ratios, etc.)

## Alerts Storage (Separate File/Database)
**Location**: `{config.WIKI_STREAM_DIR}/alerts/`

**Format**: JSON files (one file per collection session, separate from metrics)

**Schema**:
- timestamp: String - ISO timestamp of alert
- entity: String - Entity name that triggered alert
- alert_type: String - Type of alert (VANDALISM, DELETION, BLOCK, REVERT, PROTECTION, LARGE_DELETION)
- user: String - Wikipedia username
- details: String - Alert details/comment
- severity: String - Severity level (HIGH, MEDIUM, LOW)

## Metrics Calculated
1. **Edit count per entity**: How many edits per tracked entity
2. **Edit frequency over time**: Timestamps of events
3. **Most active editors**: Users grouped by edit count
4. **Change size distribution**: Distribution of page size changes
5. **Bot vs human edit ratio**: Ratio of automated vs manual edits

## Alert Conditions
1. Edit comments containing alert keywords: vandalism, deletion, block, revert, protection
2. Large deletions: More than 1000 bytes removed in a single edit
3. Anonymous users making significant changes: (future enhancement)

## Implementation Notes
- Uses Wikipedia EventStreams API: {config.WIKI_STREAM_URL}
- Events are filtered to match tracked entities by title matching
- Metrics and alerts are stored in separate files as per requirements
- In production, this would run continuously as a streaming job
"""

Path(f"{config.WIKI_STREAM_DIR}/README.md").write_text(stream_doc)
print(f"Documentation saved to: {config.WIKI_STREAM_DIR}/README.md")

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "=" * 80)
print("STREAM PROCESSING COMPLETE")
print("=" * 80)
print(f"""
Summary:
--------
✓ Selected {len(entities_to_track)} entities from IMDB dataset
✓ Collected {len(events)} events from Wikipedia EventStreams
✓ Calculated metrics (stored in: {metrics_file.name})
✓ Detected {len(alerts)} alerts (stored in: {alerts_file.name})
✓ Documentation created: {config.WIKI_STREAM_DIR}/README.md

Output Structure:
-----------------
- Metrics: {config.WIKI_STREAM_DIR}/metrics/ (JSON format)
- Alerts: {config.WIKI_STREAM_DIR}/alerts/ (JSON format, separate from metrics)

The alerts are routed to a different file/database as required by the project specifications.
""")

print("\n" + "=" * 80)
print("PROJECT COMPLETE")
print("=" * 80)
print("""
✓ All IMDB analysis questions (1-14) completed
✓ Wikipedia stream processing implemented and executed
✓ Metrics calculated and stored
✓ Alert system implemented (separate storage)
✓ All code documented with comments
✓ Output structure clearly defined
✓ Ready for submission
""")

spark.stop()


PART 2: WIKIPEDIA STREAM PROCESSING

Overview:
---------
This section implements a stream processing job that tracks Wikipedia events 
for selected IMDB entities. The system:

1. Selects 5 entities from the IMDB dataset
2. Monitors Wikipedia via EventStreams API for changes to these entities
3. Generates metrics such as edit counts, user activity, change frequency
4. Implements alerts for specific events (e.g., vandalism, deletions, blocks)
5. Stores results in structured format (JSON files)

Output Structure:
-----------------
- Metrics: stored in data/wiki_stream/metrics/ (JSON format)
- Alerts: stored in data/wiki_stream/alerts/ (JSON format, separate from metrics)

Note: This implementation collects a sample of events for demonstration.
In production, this would run continuously as a streaming job.


Selecting entities to track...

Selected 5 entities to track:
1. MOVIE: O La La (ID: tt8458418)
2. ACTOR: Kenjirô Ishimaru (ID: nm0411100)
3. GENRE: Comedy (ID: genre_comedy)
4. DIREC